In [1]:
!pip install onnxruntime
!pip install numpy
!pip install opencv-python
!pip install ultralytics


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import onnxruntime as ort

In [3]:

model = YOLO("yolov8n.pt")
model.export(format="onnx")

Ultralytics 8.3.203 🚀 Python-3.10.12 torch-2.8.0+cu128 CPU (12th Gen Intel Core(TM) i9-12900KS)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.19.0 opset 22...
ONNX: slimming with onnxslim 0.1.71...
ONNX: export success ✅ 0.6s, saved as 'yolov8n.onnx' (12.3 MB)

Export complete (0.7s)
Results saved to /home/kenny/pytorch/learn/in_and_out
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8n.onnx'

In [4]:
import requests

url = "https://videos.pexels.com/video-files/853889/853889-hd_1920_1080_25fps.mp4"
response = requests.get(url)
with open("people_walking.mp4", "wb") as f:
    f.write(response.content)

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import onnxruntime as ort
from deep_sort_realtime.deepsort_tracker import DeepSort
import math

class detect_onnx:
    @staticmethod
    def letter_box_resize(image, size=(640, 640)):
        ih, iw = image.shape[0:2]
        h, w = size
        scale = min(w / iw, h / ih)
        nw, nh = int(iw * scale), int(ih * scale)
        image_resized = cv2.resize(image, (nw, nh))
        new_image = np.full((h, w, 3), 114, dtype=np.uint8)
        top, left = (h - nh) // 2, (w - nw) // 2
        new_image[top:top + nh, left:left + nw, :] = image_resized
        return new_image, scale, top, left
    
    @staticmethod
    def scale_boxes(boxes, scale, top, left):

        x1 = (boxes[0] - left) / scale
        y1 = (boxes[1] - top) / scale
        x2 = (boxes[2] - left) / scale
        y2 = (boxes[3] - top) / scale

        return [x1, y1, x2, y2]

    @staticmethod
    def preprocess(image):

        # input_image, scale, top, left = detect_onnx.letter_box_resize(image)
        input_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        input_image = input_image.astype(np.float32)
        input_image = np.transpose(input_image, (2, 0, 1))
        input_image = np.expand_dims(input_image, axis=0)
        input_image /= 255.0
        return input_image
    
    
    @staticmethod
    def postprocess(frame,outputs,scale,top,left,conf_threshold=0.25,nms_threshold=0.35):
        detections=outputs[0]
        detections=np.squeeze(detections)
        num_detection=detections.shape[1]
        boxes=[]
        scores=[]
        class_ids=[]
        for i in range(num_detection):
            det=detections[:,i]
            class_scores=det[4:]
            class_id=np.argmax(class_scores)
            conf=class_scores[class_id]

            if class_id!=0:
                continue

            if conf<conf_threshold:
                continue

            cx,cy,w,h=det[:4]
            x1=int(cx-w/2)
            y1=int(cy-h/2)
            x2=int(cx+w/2)
            y2=int(cy+h/2)

            boxes.append([x1,y1,x2,y2])
            scores.append(float(conf))
            class_ids.append(class_id)

        nms_boxes=[(x1,y1,x2-x1,y2-y1) for (x1,y1,x2,y2) in boxes]
        indices=cv2.dnn.NMSBoxes(nms_boxes,scores,conf_threshold,nms_threshold)
        result=[]
        if len(indices)>0:
            for i in indices:
                result.append({'boxes': boxes[i], 'scores': scores[i], 'class_ids': class_ids[i]})

                # scaled_box=detect_onnx.scale_boxes(boxes[i],scale,top,left)
                # result.append({'boxes': scaled_box, 'scores': scores[i], 'class_ids': class_ids[i]})

        return result 


    @staticmethod
    def draw_boxes(image, result):
        for det in result:
            x1, y1, x2, y2 = map(int, det['boxes'])
            score = det['scores']
            class_id = det['class_ids']
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(image, f'ID:{class_id} {score:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        return image


    @staticmethod
    def infer():
        session = ort.InferenceSession(
            "yolov8n.onnx",
            providers=['TensorrtExecutionProvider','CUDAExecutionProvider', 'CPUExecutionProvider']
        )
        print("Active providers:", session.get_providers())
        cap = cv2.VideoCapture("people_walking.mp4")
        # cap = cv2.VideoCapture(0)  # Use webcam

        line_y = 315
        next_track_id = 0
        tracks = {}  # track_id: {'centroid': (x, y), 'side': 'above'/'below', 'missed': int}
        in_count = 0
        out_count = 0
        max_distance = 50  # Max pixels to consider same person
        max_missed = 5     # Frames to keep "lost" tracks

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            image_bgr, scale, top, left = detect_onnx.letter_box_resize(frame)
            input_image = detect_onnx.preprocess(image_bgr)
            inputs = {session.get_inputs()[0].name: input_image}
            outputs = session.run(None, inputs)
            result = detect_onnx.postprocess(image_bgr, outputs, scale, top, left)

            detections = []
            centroids = []
            for det in result:
                x1, y1, x2, y2 = det['boxes']
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)
                centroids.append((cx, cy))
                detections.append((x1, y1, x2, y2))

            # Track assignment
            assigned_tracks = set()
            assigned_detections = set()
            updated_tracks = {}

            for track_id, track in tracks.items():
                min_dist = float('inf')
                min_idx = -1
                for idx, centroid in enumerate(centroids):
                    if idx in assigned_detections:
                        continue
                    dist = math.hypot(track['centroid'][0] - centroid[0], track['centroid'][1] - centroid[1])
                    if dist < min_dist:
                        min_dist = dist
                        min_idx = idx
                if min_dist < max_distance:
                    assigned_tracks.add(track_id)
                    assigned_detections.add(min_idx)
                    new_centroid = centroids[min_idx]
                    x1, y1, x2, y2 = detections[min_idx]
                    center_y = new_centroid[1]
                    side = 'above' if center_y < line_y else 'below'
                    prev_side = track['side']
                    # Check crossing
                    if prev_side != side:
                        if prev_side == 'above' and side == 'below':
                            in_count += 1
                            cv2.putText(image_bgr, f"IN!", (x1, y1 - 30),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        elif prev_side == 'below' and side == 'above':
                            out_count += 1
                            cv2.putText(image_bgr, f"OUT!", (x1, y1 - 30),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    # --- Add/update history ---
                    history = track.get('history', [])
                    history.append(new_centroid)
                    if len(history) > 30:  # Limit history length
                        history = history[-30:]
                    updated_tracks[track_id] = {
                        'centroid': new_centroid,
                        'side': side,
                        'missed': 0,
                        'history': history
                    }
                    cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    cv2.putText(image_bgr, f'ID:{track_id}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                else:
                    track['missed'] += 1
                    if track['missed'] < max_missed:
                        updated_tracks[track_id] = track

            # Create new tracks for unmatched detections
            for idx, centroid in enumerate(centroids):
                if idx in assigned_detections:
                    continue
                x1, y1, x2, y2 = detections[idx]
                center_y = centroid[1]
                side = 'above' if center_y < line_y else 'below'
                updated_tracks[next_track_id] = {
                    'centroid': centroid,
                    'side': side,
                    'missed': 0,
                    'history': [centroid]  # Start history
                }
                cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 255), 2)
                cv2.putText(image_bgr, f'ID:{next_track_id}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                next_track_id += 1

            tracks = updated_tracks

            # --- Draw trajectory paths ---
            for track in tracks.values():
                history = track.get('history', [])
                if len(history) > 1:
                    pts = np.array(history, dtype=np.int32)
                    cv2.polylines(image_bgr, [pts], False, (0, 0, 255), 2)

            # Draw line and counts
            cv2.line(image_bgr, (0, line_y), (639, line_y), (255, 0, 0), 2)
            cv2.putText(image_bgr, f'IN: {in_count}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image_bgr, f'OUT: {out_count}', (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow("YOLOv8 Simple Tracker", image_bgr)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_onnx.infer()


Active providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


With deep sort


In [ ]:
# !pip install deep_sort_realtime

In [ ]:
# from ultralytics import YOLO
# import cv2
# import numpy as np
# import onnxruntime as ort
# from deep_sort_realtime.deepsort_tracker import DeepSort


# class detect_onnx:
#     @staticmethod
#     def letter_box_resize(image, size=(640, 640)):
#         ih, iw = image.shape[0:2]
#         h, w = size
#         scale = min(w / iw, h / ih)
#         nw, nh = int(iw * scale), int(ih * scale)
#         image_resized = cv2.resize(image, (nw, nh))
#         new_image = np.full((h, w, 3), 114, dtype=np.uint8)
#         top, left = (h - nh) // 2, (w - nw) // 2
#         new_image[top:top + nh, left:left + nw, :] = image_resized
#         return new_image, scale, top, left
    
#     @staticmethod
#     def scale_boxes(boxes, scale, top, left):

#         x1 = (boxes[0] - left) / scale
#         y1 = (boxes[1] - top) / scale
#         x2 = (boxes[2] - left) / scale
#         y2 = (boxes[3] - top) / scale

#         return [x1, y1, x2, y2]

#     @staticmethod
#     def preprocess(image):

#         # input_image, scale, top, left = detect_onnx.letter_box_resize(image)
#         input_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         input_image = input_image.astype(np.float32)
#         input_image = np.transpose(input_image, (2, 0, 1))
#         input_image = np.expand_dims(input_image, axis=0)
#         input_image /= 255.0
#         return input_image
    
    
#     @staticmethod
#     def postprocess(frame,outputs,scale,top,left,conf_threshold=0.25,nms_threshold=0.35):
#         detections=outputs[0]
#         detections=np.squeeze(detections)
#         num_detection=detections.shape[1]
#         boxes=[]
#         scores=[]
#         class_ids=[]
#         for i in range(num_detection):
#             det=detections[:,i]
#             class_scores=det[4:]
#             class_id=np.argmax(class_scores)
#             conf=class_scores[class_id]

#             if class_id!=0:
#                 continue

#             if conf<conf_threshold:
#                 continue

#             cx,cy,w,h=det[:4]
#             x1=int(cx-w/2)
#             y1=int(cy-h/2)
#             x2=int(cx+w/2)
#             y2=int(cy+h/2)

#             boxes.append([x1,y1,x2,y2])
#             scores.append(float(conf))
#             class_ids.append(class_id)

#         nms_boxes=[(x1,y1,x2-x1,y2-y1) for (x1,y1,x2,y2) in boxes]
#         indices=cv2.dnn.NMSBoxes(nms_boxes,scores,conf_threshold,nms_threshold)
#         result=[]
#         if len(indices)>0:
#             for i in indices:
#                 result.append({'boxes': boxes[i], 'scores': scores[i], 'class_ids': class_ids[i]})

#                 # scaled_box=detect_onnx.scale_boxes(boxes[i],scale,top,left)
#                 # result.append({'boxes': scaled_box, 'scores': scores[i], 'class_ids': class_ids[i]})

#         return result 


#     @staticmethod
#     def draw_boxes(image, result):
#         for det in result:
#             x1, y1, x2, y2 = map(int, det['boxes'])
#             score = det['scores']
#             class_id = det['class_ids']
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
#             cv2.putText(image, f'ID:{class_id} {score:.2f}', (x1, y1 - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
#         return image

#     @staticmethod
#     def infer():
#         session = ort.InferenceSession(
#             "yolov8n.onnx",
#             providers=['TensorrtExecutionProvider','CUDAExecutionProvider', 'CPUExecutionProvider']
#         )
#         print("Active providers:", session.get_providers())
#         cap = cv2.VideoCapture("people_walking.mp4")
#         # cap = cv2.VideoCapture(0)  # Use webcam
#         tracker = DeepSort(max_age=30)  # Initialize Deep SORT

#         line_y = 315
#         track_history = {}  # track_id: list of (cx, cy)
#         last_side = {}      # track_id: last_side ('above' or 'below')
#         in_count = 0
#         out_count = 0
#         min_move = 5  # Minimum pixels moved to consider as crossing

#         while True:
#             ret, frame = cap.read()
#             if not ret:
#                 break

#             image_bgr, scale, top, left = detect_onnx.letter_box_resize(frame)
#             input_image = detect_onnx.preprocess(image_bgr)
#             inputs = {session.get_inputs()[0].name: input_image}
#             outputs = session.run(None, inputs)
#             result = detect_onnx.postprocess(image_bgr, outputs, scale, top, left)

#             detections = []
#             for det in result:
#                 x1, y1, x2, y2 = det['boxes']
#                 conf = det['scores']
#                 detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

#             tracks = tracker.update_tracks(detections, frame=image_bgr)

#             for track in tracks:
#                 if not track.is_confirmed():
#                     continue
#                 track_id = track.track_id
#                 ltrb = track.to_ltrb()
#                 x1, y1, x2, y2 = map(int, ltrb)
#                 center_x = int((x1 + x2) / 2)
#                 center_y = int((y1 + y2) / 2)

#                 # --- Update trajectory history ---
#                 if track_id not in track_history:
#                     track_history[track_id] = []
#                 track_history[track_id].append((center_x, center_y))
#                 if len(track_history[track_id]) > 30:
#                     track_history[track_id] = track_history[track_id][-30:]

#                 # Determine which side of the line the center is on
#                 side = 'above' if center_y < line_y else 'below'

#                 if track_id in last_side:
#                     prev_side = last_side[track_id]
#                     # Check if side changed (person crossed the line)
#                     if prev_side != side:
#                         if prev_side == 'above' and side == 'below':
#                             in_count += 1
#                             cv2.putText(image_bgr, f"IN!", (x1, y1 - 30),
#                                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
#                             print(f"Track {track_id} crossed IN. Total IN: {in_count}")
#                         elif prev_side == 'below' and side == 'above':
#                             out_count += 1
#                             cv2.putText(image_bgr, f"OUT!", (x1, y1 - 30),
#                                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
#                             print(f"Track {track_id} crossed OUT. Total OUT: {out_count}")
#                 last_side[track_id] = side

#                 cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 255), 2)
#                 cv2.putText(image_bgr, f'ID:{track_id}', (x1, y1 - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

#             # --- Draw trajectory for each track ---
#             for track_id, history in track_history.items():
#                 if len(history) > 1:
#                     pts = np.array(history, dtype=np.int32)
#                     cv2.polylines(image_bgr, [pts], False, (0, 0, 255), 2)

#             # Draw line and counts
#             cv2.line(image_bgr, (0, line_y), (639, line_y), (255, 0, 0), 2)
#             cv2.putText(image_bgr, f'IN: {in_count}', (10, 30),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#             cv2.putText(image_bgr, f'OUT: {out_count}', (10, 70),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
#             cv2.imshow("ONNX YOLOv8 + Deep SORT", image_bgr)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

#         cap.release()
#         cv2.destroyAllWindows()

# if __name__ == "__main__":
#     detect_onnx.infer()

